In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
272,AFG,Asia,Afghanistan,2020-09-28,39227.0,0.0,26.143,1453.0,0.0,1.714,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
476,ALB,Europe,Albania,2020-09-28,13259.0,106.0,124.857,377.0,2.0,2.143,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
749,DZA,Africa,Algeria,2020-09-28,51067.0,153.0,177.286,1714.0,3.0,6.000,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-27,France,542028.0,0.0,31831.0,0.0,0.0,0.0
0,2020-09-27,China,90475.0,0.0,4744.0,0.0,0.0,0.0
0,2020-09-27,Italy,309746.0,0.0,35853.0,0.0,0.0,0.0
0,2020-09-27,Spain,725642.0,0.0,31324.0,0.0,0.0,0.0
0,2020-09-27,United States,7151002.0,0.0,205693.0,0.0,0.0,0.0
0,2020-09-27,World,33411092.0,0.0,1003788.0,0.0,0.0,0.0
0,2020-09-27,United Kingdom,434399.0,0.0,42027.0,0.0,0.0,0.0
0,2020-09-27,Germany,286307.0,0.0,9470.0,0.0,0.0,0.0
0,2020-09-27,Iran,446209.0,0.0,25610.0,0.0,0.0,0.0
0,2020-09-27,Turkey,314273.0,0.0,7989.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-28,France,542028.0,538569.0,31831.0,31727.0,3459.0,104.0
0,2020-09-28,China,90475.0,90483.0,4744.0,4739.0,-8.0,5.0
0,2020-09-28,Italy,309746.0,309870.0,35853.0,35835.0,-124.0,18.0
0,2020-09-27,Spain,725642.0,716481.0,31324.0,31232.0,9161.0,92.0
0,2020-09-28,United States,7151002.0,7115046.0,205693.0,204756.0,35956.0,937.0
0,2020-09-28,World,33411092.0,33178019.0,1003788.0,998784.0,233073.0,5004.0
0,2020-09-28,United Kingdom,434399.0,434969.0,42027.0,41988.0,-570.0,39.0
0,2020-09-28,Germany,286307.0,285332.0,9470.0,9460.0,975.0,10.0
0,2020-09-28,Iran,446209.0,446448.0,25610.0,25589.0,-239.0,21.0
0,2020-09-28,Turkey,314273.0,314433.0,7989.0,7997.0,-160.0,-8.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")